## Perubahan Struktur Dataset

### Perubahan Struktur Dataset Pembelian

In [ ]:
import pandas as pd
import re
from pathlib import Path

def parse_number(x):
    """Konversi string angka format Indonesia menjadi float"""
    if pd.isna(x) or str(x).strip() == '':
        return 0.0
    x = str(x).replace('.', '').replace(',', '.')
    try:
        return float(x)
    except:
        return 0.0

# Lokasi file
file_path = Path("pembelian.tsv")

# Baca file baris per baris
with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

data = []
kode, nama, unit = '', '', ''

for line in lines:
    line = line.strip()
    if not line:
        continue

    # Deteksi baris header produk (contoh: A000001 ANATON TAB STRIP)
    header_match = re.match(r'^([A-Z0-9]+)\s+(.+?)\s+(STRIP|BTL|BOX|PCS)$', line)
    if header_match:
        kode = header_match.group(1).strip()
        nama = header_match.group(2).strip()
        unit = header_match.group(3).strip()
        continue

    # Deteksi baris transaksi (dimulai tanggal DD-MM-YY)
    date_match = re.match(r'^(\d{2}-\d{2}-\d{2})', line)
    if date_match:
        parts = re.split(r'\s{2,}', line)
        tanggal = parts[0]
        no_transaksi = parts[1] if len(parts) > 1 else ''

        qty_msk = nilai_msk = qty_klr = nilai_klr = 0.0

        # Format umum: tanggal | no_transaksi | qty | nilai
        if len(parts) == 4:
            # Cek kode transaksi (1. = masuk, 2. = keluar)
            if no_transaksi.strip().startswith('1.'):
                qty_msk = parse_number(parts[2])
                nilai_msk = parse_number(parts[3])
            elif no_transaksi.strip().startswith('2.'):
                qty_klr = parse_number(parts[2])
                nilai_klr = parse_number(parts[3])

        elif len(parts) == 5:
            # Bisa terjadi karena format tidak konsisten, kita cek pola juga
            if no_transaksi.strip().startswith('1.'):
                qty_msk = parse_number(parts[2])
                nilai_msk = parse_number(parts[3])
            elif no_transaksi.strip().startswith('2.'):
                qty_klr = parse_number(parts[2])
                nilai_klr = parse_number(parts[3])

        elif len(parts) >= 6:
            # Kalau keduanya ada (jarang terjadi)
            qty_msk = parse_number(parts[2])
            nilai_msk = parse_number(parts[3])
            qty_klr = parse_number(parts[4])
            nilai_klr = parse_number(parts[5])

        # Tentukan kategori dan total
        if qty_msk > 0:
            kategori = 'MASUK'
            qty_total = qty_msk
            nilai_total = nilai_msk
        elif qty_klr > 0:
            kategori = 'KELUAR'
            qty_total = qty_klr
            nilai_total = nilai_klr
        else:
            kategori = 'LAINNYA'
            qty_total = 0
            nilai_total = 0

        data.append({
            'KODE': kode,
            'NAMA_PRODUK': nama,
            'UNIT': unit,
            'TANGGAL': pd.to_datetime(tanggal, format='%d-%m-%y', errors='coerce'),
            'NO_TRANSAKSI': no_transaksi,
            'QTY_MSK': qty_msk,
            'NILAI_MSK': nilai_msk,
            'QTY_KLR': qty_klr,
            'NILAI_KLR': nilai_klr,
            'KATEGORI': kategori,
            'QTY_TOTAL': qty_total,
            'NILAI_TOTAL': nilai_total
        })

# Buat DataFrame
df = pd.DataFrame(data)

# Urutkan berdasarkan kode dan tanggal
df = df.sort_values(by=["KODE", "TANGGAL"]).reset_index(drop=True)

# Simpan hasil ke CSV
output_path = "pembelian_fix.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ File '{output_path}' berhasil dibuat!")
print(df.head(15))


✅ File 'pembelian.csv' berhasil dibuat!
       KODE NAMA_PRODUK   UNIT    TANGGAL          NO_TRANSAKSI  QTY_MSK  \
0   A000001  ANATON TAB  STRIP 2021-07-06  1.13-210706.0908-003     10.0   
1   A000001  ANATON TAB  STRIP 2021-07-12   2.6-210712.1519-097      0.0   
2   A000001  ANATON TAB  STRIP 2021-07-12  2.11-210712.1633-013      0.0   
3   A000001  ANATON TAB  STRIP 2021-07-12  2.13-210712.1807-013      0.0   
4   A000001  ANATON TAB  STRIP 2021-07-12  2.11-210712.1855-018      0.0   
5   A000001  ANATON TAB  STRIP 2021-07-12  2.11-210712.1925-027      0.0   
6   A000001  ANATON TAB  STRIP 2021-07-12  2.11-210712.1957-035      0.0   
7   A000001  ANATON TAB  STRIP 2021-07-12   2.6-210712.0907-023      0.0   
8   A000001  ANATON TAB  STRIP 2021-07-13  2.11-210713.1102-011      0.0   
9   A000001  ANATON TAB  STRIP 2021-07-13   2.6-210713.1701-006      0.0   
10  A000001  ANATON TAB  STRIP 2021-08-23  1.12-210823.1955-003     10.0   
11  A000001  ANATON TAB  STRIP 2021-09-20  2.11-

### Perubahan Struktur Dataset Stok

In [ ]:
import pandas as pd
import re
from pathlib import Path

# === Fungsi bantu untuk parsing angka format Indonesia ===
def parse_number(x):
    """Konversi string angka format Indonesia menjadi float"""
    if pd.isna(x) or str(x).strip() == '':
        return 0.0
    x = str(x).replace('.', '').replace(',', '.')
    try:
        return float(x)
    except:
        return 0.0

# === Lokasi file input ===
file_path = Path("stok.tsv")

# Baca file baris per baris
with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

data = []

for line in lines:
    line = line.strip()
    if not line:
        continue

    # Lewati baris header
    if line.startswith("KODE") or line.startswith("Kode"):
        continue

    # Pisahkan berdasarkan 2+ spasi
    parts = re.split(r'\s{2,}', line)
    # Contoh hasil split:
    # ['A000001', 'ANATON TAB', 'ETL1', '12,00', 'STRIP']

    if len(parts) >= 5:
        kode = parts[0].strip()
        nama_produk = parts[1].strip()
        lokasi = parts[2].strip()
        qty_stok = parse_number(parts[3])
        unit = parts[4].strip()

        data.append({
            'KODE': kode,
            'NAMA_PRODUK': nama_produk,
            'LOKASI': lokasi,
            'QTY_STOK': qty_stok,
            'UNIT': unit
        })

# Buat DataFrame
df = pd.DataFrame(data)

# Urutkan berdasarkan KODE
df = df.sort_values(by=["KODE"]).reset_index(drop=True)

# Simpan ke CSV
output_path = "stok_fix.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ File '{output_path}' berhasil dibuat!")
print(df.head(10))


✅ File 'stok.csv' berhasil dibuat!
      KODE         NAMA_PRODUK LOKASI  QTY_STOK   UNIT
0  A000001          ANATON TAB   ETL1      12.0  STRIP
1   A00001       ACTIVED HIJAU  ETL3A       2.0    BTL
2  A000012  APIALYS SYR 100 ML  ETL3A       2.0    BTL
3  A000014     ALKOHOL 1000 ML  ETL3B       7.0    BTL
4  A000016     ALLOPURINOL 300   RAK2      40.0  STRIP
5  A000018   ATORVASTATIN 10MG   RAK2       6.0  STRIP
6   A00004     ACYCLOVIR 200MG   RAK2      13.0  STRIP
7  A000040         MEFIX 500MG   RAK1       9.0  STRIP
8   A00005     ACYCLOVIR 400MG   RAK2      21.0  STRIP
9  A000066       ANDALAN KB FE   RAK4      35.0  STRIP


## Ekstraksi data pembelian menjadi pembelian dan penjualan

In [4]:
# --- CELL 1: Impor Library ---
import pandas as pd

# Mengatur opsi tampilan pandas agar lebih mudah dibaca
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [5]:
# --- CELL 2: Muat Dataset Utama ---
# Membaca file 'pembelian_fix.csv' yang sudah Anda berikan.

file_pembelian_utama = 'pembelian_fix.csv'

try:
    df_main = pd.read_csv(file_pembelian_utama)
    print(f"Dataset '{file_pembelian_utama}' berhasil dimuat.")
    print(f"Jumlah baris total: {len(df_main)}")
    
    # Menampilkan info untuk memastikan kolom terbaca dengan benar
    print("\n--- Info Data Asli ---")
    df_main.info()

except FileNotFoundError:
    print(f"Error: File '{file_pembelian_utama}' tidak ditemukan.")
except Exception as e:
    print(f"Terjadi error saat membaca file: {e}")

Dataset 'pembelian_fix.csv' berhasil dimuat.
Jumlah baris total: 138364

--- Info Data Asli ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138364 entries, 0 to 138363
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   KODE          138364 non-null  object 
 1   NAMA_PRODUK   138364 non-null  object 
 2   UNIT          138364 non-null  object 
 3   TANGGAL       138364 non-null  object 
 4   NO_TRANSAKSI  138364 non-null  object 
 5   QTY_MSK       138364 non-null  float64
 6   NILAI_MSK     138364 non-null  float64
 7   QTY_KLR       138364 non-null  float64
 8   NILAI_KLR     138364 non-null  float64
 9   KATEGORI      138364 non-null  object 
 10  QTY_TOTAL     138364 non-null  float64
 11  NILAI_TOTAL   138364 non-null  float64
dtypes: float64(6), object(6)
memory usage: 12.7+ MB


In [6]:
# --- CELL 3: Pisahkan Data Menjadi Pembelian dan Penjualan ---
# Menerapkan logika pemisahan ke df_main

# 1. Buat DataFrame Pembelian (Kategori MASUK)
# Filter baris di mana KATEGORI adalah 'MASUK'
# Kita juga akan membersihkan kolom string KATEGORI untuk jaga-jaga jika ada spasi
df_pembelian = df_main[df_main['KATEGORI'].str.strip() == 'MASUK'].copy()

# Hapus kolom QTY_KLR dan NILAI_KLR 
if 'QTY_KLR' in df_pembelian.columns and 'NILAI_KLR' in df_pembelian.columns:
    df_pembelian = df_pembelian.drop(columns=['QTY_KLR', 'NILAI_KLR'])
else:
    print("Peringatan: Kolom QTY_KLR atau NILAI_KLR tidak ditemukan di df_pembelian.")


# 2. Buat DataFrame Penjualan (Kategori KELUAR)
# Filter baris di mana KATEGORI adalah 'KELUAR'
df_penjualan = df_main[df_main['KATEGORI'].str.strip() == 'KELUAR'].copy()

# Hapus kolom QTY_MSK dan NILAI_MSK
if 'QTY_MSK' in df_penjualan.columns and 'NILAI_MSK' in df_penjualan.columns:
    df_penjualan = df_penjualan.drop(columns=['QTY_MSK', 'NILAI_MSK'])
else:
    print("Peringatan: Kolom QTY_MSK atau NILAI_MSK tidak ditemukan di df_penjualan.")

print("\nData berhasil dipisahkan berdasarkan KATEGORI.")


Data berhasil dipisahkan berdasarkan KATEGORI.


In [7]:
# --- CELL 4: Tampilkan Hasil ---

print(f"\n--- Hasil DataFrame Pembelian (Total: {len(df_pembelian)} baris) ---")
print(df_pembelian.head())

print(f"\n--- Hasil DataFrame Penjualan (Total: {len(df_penjualan)} baris) ---")
print(df_penjualan.head())


--- Hasil DataFrame Pembelian (Total: 15348 baris) ---
         KODE NAMA_PRODUK   UNIT     TANGGAL          NO_TRANSAKSI  QTY_MSK  NILAI_MSK KATEGORI  QTY_TOTAL  NILAI_TOTAL
0     A000001  ANATON TAB  STRIP  2021-07-06  1.13-210706.0908-003     10.0    2520.00    MASUK       10.0      2520.00
10    A000001  ANATON TAB  STRIP  2021-08-23  1.12-210823.1955-003     10.0    3139.15    MASUK       10.0      3139.15
12    A000001  ANATON TAB  STRIP  2021-10-13  1.11-211013.1609-006     10.0    3186.98    MASUK       10.0      3186.98
20  A00000156          zz    BTL  2021-03-25  1.13-210325.2028-006      4.0   14355.43    MASUK        4.0     14355.43
24  A00000157           z    BTL  2021-03-25  1.13-210325.2028-006      3.0   20295.60    MASUK        3.0     20295.60

--- Hasil DataFrame Penjualan (Total: 120511 baris) ---
      KODE NAMA_PRODUK   UNIT     TANGGAL          NO_TRANSAKSI  QTY_KLR  NILAI_KLR KATEGORI  QTY_TOTAL  NILAI_TOTAL
1  A000001  ANATON TAB  STRIP  2021-07-12   2.6-21

In [8]:
# --- CELL: Ekspor DataFrame ke File CSV ---

# Tentukan nama file untuk ekspor
file_pembelian_baru = 'data_pembelian_final.csv'
file_penjualan_baru = 'data_penjualan_final.csv'

try:
    # Ekspor df_pembelian ke CSV
    # index=False digunakan agar nomor baris (index) pandas tidak ikut disimpan
    df_pembelian.to_csv(file_pembelian_baru, index=False)
    
    # Ekspor df_penjualan ke CSV
    df_penjualan.to_csv(file_penjualan_baru, index=False)
    
    print("Ekspor berhasil!")
    print(f"File '{file_pembelian_baru}' telah disimpan.")
    print(f"File '{file_penjualan_baru}' telah disimpan.")

except NameError:
    print("Error: Pastikan Anda sudah menjalankan sel sebelumnya untuk membuat 'df_pembelian' dan 'df_penjualan'.")
except Exception as e:
    print(f"Terjadi error saat mengekspor file: {e}")

Ekspor berhasil!
File 'data_pembelian_final.csv' telah disimpan.
File 'data_penjualan_final.csv' telah disimpan.
